# TomoBase Tutorial

This notebook is designed to show the basic processes of how to use the tomobase library for running tomographic processes. To do so each cell in this library outlines how to perform a procedure with the library:

Table of Contents:
[t](#data-imports)


## Introduction

## 

In [1]:
import os
os.environ['QT_API'] = 'pyqt5'

import tomobase
import pandas as pd
from tomobase import data
from tomobase.processes import alignments
import ncempy.io as nio
import numpy as np 
import astra
print(astra.use_cuda())
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sino_inc = data.Sinogram.from_file('aunpinc_aligned.mrc')
sino_bin = data.Sinogram.from_file('aunpbin_aligned.mrc')
sino_grs = data.Sinogram.from_file('aunpgrs_aligned.mrc')


#sino_inc.data = np.delete(sino_inc.data, 0, axis=2)


ERROR:root:hyperspy module not found.


True
{'dimensions': array([512, 512,  65]), 'MRCtype': 2, 'compressor': None, 'dtype': '<f4', 'pixelsize': array([3.87999993e-10, 3.87999993e-10, 3.88000005e-10]), 'pixelunits': '\\AA', 'minImage': -2171.297, 'maxImage': 19445.91, 'meanImage': 8637.307, 'extendedBytes': 1712, 'metaId': b'json', 'voltage': 0.0, 'C3': 0.0, 'gain': 1.0, 'packedBytes': (0,), 'angles': [-64.0, -62.0, -60.0, -58.0, -56.0, -54.0, -52.0, -50.0, -48.0, -46.0, -44.0, -42.0, -40.0, -38.0, -36.0, -34.0, -32.0, -30.0, -28.0, -26.0, -24.0, -22.0, -20.0, -18.0, -16.0, -14.0, -12.0, -10.0, -8.0, -6.0, -4.0, -2.0, 0.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 17.99, 19.99, 21.99, 24.0, 26.0, 28.0, 30.0, 32.0, 34.0, 36.0, 37.99, 40.0, 42.0, 44.0, 46.0, 47.99, 50.0, 52.0, 54.0, 56.0, 58.0, 60.0, 62.0, 64.0], 'times': [11.866396768297136, 178.18059555260749, 240.6228149361412, 274.26372651760715, 314.84597075360733, 358.1849991929932, 386.8964295558288, 417.2444327678159, 437.24698095965294, 459.97408845328715, 503.425

In [7]:
name = ['inc', 'bin', 'grs']
sinos = [sino_inc, sino_bin, sino_grs]
rec_weighted = []
rec = []
for i, sino in enumerate(sinos):
    rec.append(tomobase.processes.reconstruct_weighted_sirt(sino, iterations=100, weighted = False))
    rec[i].to_file(f'aunp_{name[i]}_rec.rec')
    print('reconstruct', i)
    rec_weighted.append(tomobase.processes.reconstruct_weighted_sirt(sino, iterations=100, weighted = True))
    rec_weighted[i].to_file(f'aunp_{name[i]}_rec_weighted.rec')
    print('reconstruct weighted', i)

reconstruct 0
65
reconstruct weighted 0
reconstruct 1
64
reconstruct weighted 1
reconstruct 2
62
reconstruct weighted 2


In [6]:
tomobase.plots.Imagine(*sinos)

In [ ]:
sino_inc.data = np.concatenate((sino_inc.data, img2.getDataset(0)[0][:,:,None]), axis=2)
sino_inc.angles = np.append(sino_inc.angles,64)
sino_inc.angles = np.delete(sino_inc.angles, 0)
print(len(sino_inc.angles))

sino_grs.angles = np.delete(sino_grs.angles, -1)
print(len(sino_grs.angles))

sino_bin.angles = np.delete(sino_bin.angles, -1)
print(len(sino_bin.angles))


In [ ]:
sino_inc = alignments.align_sinogram_center_of_mass(sino_inc)
sino_inc = alignments.align_sinogram_xcorr(sino_inc)
sino_inc = alignments.background_subtract_median(sino_inc)
sino_inc.show()

sino_bin = alignments.align_sinogram_center_of_mass(sino_bin)
sino_bin = alignments.align_sinogram_xcorr(sino_bin)
sino_bin = alignments.background_subtract_median(sino_bin)
sino_bin.show()


sino_grs = alignments.align_sinogram_center_of_mass(sino_grs)
sino_grs = alignments.align_sinogram_xcorr(sino_grs)
sino_grs = alignments.background_subtract_median(sino_grs)
sino_grs.show()

In [ ]:
widget = tomobase.processes.alignments.TranslateSinogramManual(sino_inc)

In [ ]:
pixel_size_low = 5.46
df = pd.DataFrame(columns=['projections', 'times', 'dosage'])
sino_temp = sino_inc
for i in range(len(sino_temp.angles)):
    time = (i+1)*2*(10**-6)*1024*1024
    area = (1024*pixel_size)**2
    C = 1.6*(10**-19)
    I = 50*(10**-12)
    D = (I*time)/(C*area)
    time2 = sino_temp.metadata['times'][i]-time
    area2 = (1024*pixel_size_low)**2
    D2 = (I*time2)/(C*area2)
    D+=D2
    
    df2 = pd.DataFrame({'projections': [i], 'times': [time], 'dosage': [D]})
    df = pd.concat([df, df2], ignore_index=True)
    
display(df)
    
    

In [ ]:
sino_temp = sino_bin
angle_diff = np.diff(sino_temp.angles)
mean = np.mean(np.abs(angle_diff))
sd = np.std(np.abs(angle_diff))
print(mean, sd)

  


In [ ]:

obj = alignments.MaskBackground(sino_inc)


In [ ]:
from tomobase import processes
import tomobase.processes.alignments

sino_inc = tomobase.processes.alignments.align_tilt_axis_shift(sino_inc)

In [ ]:
sino2 = obj.sino  
sino_inc.show()

In [ ]:
sino_inc = processes.alignments.bin(sino_inc)
rec = processes.reconstruct(sino_inc, 'sirt', iterations=100)
rec.to_file('AuNP-Incremental.rec')